# 🎼 AI Music Generator Using Classical MIDI (Internship Task)
This notebook builds an LSTM-based model to generate classical music using MIDI files.
- Dataset: [Classical Music MIDI Dataset on Kaggle](https://www.kaggle.com/datasets/soumikrakshit/classical-music-midi)
- Libraries: music21, Keras, TensorFlow

In [ ]:
# 🛠️ Install Required Libraries
!pip install music21 keras tensorflow

In [ ]:
# 📁 Import Libraries
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.utils import to_categorical

In [ ]:
# 📥 Extract Notes from MIDI
def extract_notes_from_midi(midi_dir):
    notes = []
    for file in os.listdir(midi_dir):
        if file.endswith('.mid'):
            try:
                midi = converter.parse(os.path.join(midi_dir, file))
                parts = instrument.partitionByInstrument(midi)
                elements = parts.parts[0].recurse() if parts else midi.flat.notes
                for element in elements:
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))
            except Exception as e:
                print(f"Error parsing {file}: {e}")
    return notes

# Replace with your actual path to the MIDI folder
midi_folder = './classical-music-midi/MIDI'
notes = extract_notes_from_midi(midi_folder)
print(f"Total notes extracted: {len(notes)}")

In [ ]:
# 🔢 Prepare Sequences
sequence_length = 100
pitch_names = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(pitch_names)}

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(pitch_names)

X = np.reshape(network_input, (n_patterns, sequence_length, 1)) / float(n_vocab)
y = to_categorical(network_output)

In [ ]:
# 🧠 Build the LSTM Model
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

In [ ]:
# 🚀 Train the Model
model.fit(X, y, epochs=50, batch_size=64)
# model.save('music_model.h5')

In [ ]:
# 🎶 Generate Music
int_to_note = {number: note for note, number in note_to_int.items()}
start = np.random.randint(0, len(network_input) - 1)
pattern = network_input[start]
generated_notes = []

for note_index in range(500):
    input_seq = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
    prediction = model.predict(input_seq, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_notes.append(result)
    pattern.append(index)
    pattern = pattern[1:]

In [ ]:
# 💾 Convert to MIDI and Save
def create_midi(prediction_output, filename="output.mid"):
    output_notes = []
    for pattern in prediction_output:
        if '.' in pattern or pattern.isdigit():
            notes_in_chord = [note.Note(int(n)) for n in pattern.split('.')]
            new_chord = chord.Chord(notes_in_chord)
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            output_notes.append(new_note)
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)

create_midi(generated_notes, "generated_classical.mid")
print("✅ MIDI File Saved: generated_classical.mid")